# Gentle introduction to Ray datasets APIs

© 2019-2022, Anyscale. All Rights Reserved

### Overview

As a native library and built atop Ray, Ray data allows you to exchange data among Ray tasks, actors, libraries, and applications. It also allows you to read/write training data from different file sources and storage, including csv, binary, parquet, text, etc., and supporting myriad [file formats and data sources](https://docs.ray.io/en/latest/data/dataset.html#datasource-compatibility).
<img src="images/dataset.png" width="75%" height="45%">

Using distributed [Apache Arrow](https://arrow.apache.org/), Ray dataset is designed to load and preprocess data for distributed ML training pipelines. Datasets are flexible (e.g., you can express higher-quality per-epoch global shuffles) and provides higher overall performance. Additionally, datasets comes with standard and simple transformations like `map`, `filter`, and `partition`. 

Ray datasets is not a replacement for a full-fledged data processing library for doing exploratory data analysis (EDA), extract, transform and load (ETL) or a subsitute for Apache Spark or Dask or Pandas DataFrames. Its primary objective is the last-mile rudimentary distributed data preprocessing and data ingestion for distributed ML training by providing simple primitive transformational functions.

### Learning objectives

In this introductory tutorial you will learn to:
 * what are Ray datasets, why use them and when
 * create, transform, read, and save Ray datasets
 * use shards for parallel processing of large datasets
 * understand datapipelines and their merits
 * use `DatasetPipeline` for parallel computation 
 * use datasets for last-mile ML ingestion for distributed training

### Key concepts

To work with Ray Datasets, you need to understand how `Datasets` and `Dataset Pipelines` work. That is, how datasets are stored internally and in what format. And what benefit does `Datapipelines` offer for faster processing and execution. A quick peek into each of these will shed some light into overall benefits of Ray Datasets.

Let's start with the internal format. 

#### Ray Datasets

A Ray dataset implements a distributed [Apache Arrow](https://arrow.apache.org/). As such, a Dataset consists of a list of Ray object references to blocks. Each block holds a set of items in either an [Arrow table](https://arrow.apache.org/docs/python/data.html#tables) (when created from or transformed to tabular or tensor data), a Pandas DataFrame (when created from or transformed to Pandas data), or a Python list (otherwise).

<img src="images/dataset-arch.png" width="70%" height="35%">

#### Dataset Pipelines
Datasets execute their transformations synchronously or eagerly in blocking calls. However, it can be useful to overlap dataset computations with output. This can be done with a `DatasetPipeline`.

A `DatasetPipeline` is a unified iterator over a (potentially infinite) sequence of Ray Datasets, each of which represents a window over the original data. Conceptually, it is similar to a `Spark DStream`, but manages execution over a bounded amount of source data instead of an unbounded stream. Ray computes each dataset window on-demand and stitches their output together into a single logical data iterator. `DatasetPipeline` implements most of the same transformation and output methods as Datasets (e.g., `map`, `filter`, `split`, `iter_rows`, `to_torch`, etc.).

<img src="images/ray_dataset_pipeline.jpg" width="75%" height="45%">

### Datasets Execution Model
In this section, we briefly discuss the execution model of Datasets, which may be useful for understanding and tuning performance.

#### Reading Data
Datasets uses Ray tasks, for parallelism, to read data from remote storage or source. When reading from a file-based datasource (e.g., S3, GCS), it creates a number of parallel
read tasks equal to the specified read parallelism (200 by default). One or more files will be assigned to each read task. Each read task reads its assigned files and produces one or more output blocks (Ray objects):

<img src="https://docs.ray.io/en/master/_images/dataset-read.svg" height="25%" width="50%">

In the common case, each read task produces a single output block. Read tasks may split the output into multiple blocks if the data exceeds the target max block size (2GiB by default). This automatic block splitting avoids out-of-memory errors when reading very large single files (e.g., a 100-gigabyte CSV file). All of the built-in datasources except for JSON currently support automatic block splitting.

#### Deferred Read Task Execution

When a Dataset is created using `ray.data.read_*`, only the first read task will be executed initially. This avoids blocking Dataset creation on the reading of all data files, enabling inspection functions like `ds.schema()` without incurring high read costs. `<ray.data.Dataset.schema>`() and `ds.show()` can be used right away. Executing further transformations on the Dataset will trigger execution of all read tasks.

#### Dataset Transforms

Datasets use either Ray tasks or Ray actors to transform datasets (i.e., for `ds.map_batches()`, `ds.map()`, or `ds.flat_map()`). By default, tasks are used `(compute="tasks")`. Actors can be specified with `compute="actors"`, in which case an autoscaling pool of Ray actors will be used to apply transformations. Using actors allows for expensive state initialization (e.g., for GPU-based tasks) to be re-used. Whichever compute strategy is used, each map task generally takes in one block and produces one or more output blocks. The output block splitting rule is the same as for file reads (blocks are split after hitting the target max block size of 2GiB):

<img src="https://docs.ray.io/en/master/_images/dataset-map.svg" height="25%" width="50%">

#### Shuffling Data

Certain operations like `ds.sort()` and `ds.groupby()` require data blocks to be partitioned by value. Datasets executes this in three phases. 

First, a wave of sampling tasks determines suitable partition boundaries based on a random sample of data. Second, map tasks divide each input block into a number of output blocks equal to the number of reduce tasks. Third, reduce tasks take assigned output blocks from each map task and combines them into one block. (Overall, this strategy generates O(n^2) intermediate objects where n is the number of input blocks.)

You can also change the partitioning of a Dataset using `ds.random_shuffle()` or `ds.repartition()`. The former should be used if you want to randomize the order of elements in the dataset. The second should be used if you only want to equalize the size of the Dataset blocks (e.g., after a read or transformation that may skew the distribution of block sizes). Note that repartition has two modes, `shuffle=False`, which performs the minimal data movement needed to equalize block sizes, and `shuffle=True`, which performs a full (non-random) distributed shuffle:

<img src="https://docs.ray.io/en/master/_images/dataset-shuffle.svg" height="25%" width="50%">

#### Fault tolerance

Datasets relies on task-based 🤹‍♀️ [fault tolerance](https://docs.ray.io/en/latest/ray-core/tasks/fault-tolerance.html) in Ray core. Specifically, a `Dataset` will be automatically recovered by Ray in case of failures. This works through **lineage reconstruction**: a Dataset is a collection of Ray objects stored in shared memory, and if any of these objects are lost, then Ray will recreate them by re-executing the task(s) that created them.

There are a few cases that are not currently supported: 

 1. If the original creator of the Dataset dies ☠️. This is because the creator stores the metadata for the objects that comprise the Dataset. 
 2. For a `DatasetPipeline.split()` 👯‍♂️, we do not support recovery for a consumer failure. When there are multiple consumers, they must all read the split pipeline in lockstep. To recover from this case, the pipeline and all consumers must be restarted together. 
 3. The `compute=actors`🧑‍🏭 option for transformations.

#### Execution and Memory Management

See [Execution and Memory Management](https://docs.ray.io/en/master/data/memory-management.html#data-advanced) for more details about how Datasets manages memory and optimizations such as lazy vs eager execution.

In [1]:
import logging, os, random, warnings
import ray

In [2]:
warnings.filterwarnings("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
if ray.is_initialized:
    ray.shutdown()
ray.init(logging_level=logging.ERROR)

Python version:,3.8.13
Ray version:,3.0.0.dev0
Dashboard:,http://127.0.0.1:8271


### Creating a simple Ray Dataset

For quick illustration, let's create a generic dataset of 1K integers and look at the schema and underlying datatype. [Ray Data API Documentation](https://docs.ray.io/en/latest/data/package-ref.html).

In [5]:
ds = ray.data.range(1000)
ds.count()

1000

In [6]:
ds.schema()

int

The difference between `show` and `take` is that the former takes one item at time and prints it, while the latter iterates over row items from the dataset, appends to a list and returns it. Underneath, `ds.show()` calls `ds.take()`.

In [7]:
ds.show(5)

0
1
2
3
4


In [8]:
ds.take(5)

[0, 1, 2, 3, 4]

### Creating a large Ray Dataset

Let's create a synthetic dataset, *Homeowners*, of Arrow records (800K) with several columns and data associated with it. 

To illustrate some simple transformational functions, we'll use this generated data.

In [9]:
NUM_ROWS = 800_001
STATES = ["CA", "AZ", "OR", "WA", "TX", "UT", "NV", "NM"]
M_STATUS = ["married", "single", "domestic", "divorced", "undeclared"]
GENDER = ["F", "M", "U"]
HOME_OWNER = ["condo", "house", "rental", "cottage"]

items = [{"id": i,
          "ssn": None,
          "name": None,
          "amount": i * 1.5, 
          "interest": random.randint(1,5) * .1,
          "state": random.choice(STATES),
          "marital_status": random.choice(M_STATUS),
          "property": random.choice(HOME_OWNER),
          "dependents": random.randint(1, 5),
          "defaulted": random.randint(0,1),
          "gender":random.choice(GENDER) } for i in range(1,NUM_ROWS)]
items[:2]

[{'id': 1,
  'ssn': None,
  'name': None,
  'amount': 1.5,
  'interest': 0.30000000000000004,
  'state': 'OR',
  'marital_status': 'single',
  'property': 'condo',
  'dependents': 5,
  'defaulted': 0,
  'gender': 'F'},
 {'id': 2,
  'ssn': None,
  'name': None,
  'amount': 3.0,
  'interest': 0.30000000000000004,
  'state': 'WA',
  'marital_status': 'single',
  'property': 'rental',
  'dependents': 3,
  'defaulted': 1,
  'gender': 'F'}]

#### Creating a dataset from list of dictionary items

Ray data can be created of a dictionary of items. 

*Questions: how is num_blocks computed?*

The default number of blocks is 200, just like in Spark, the default number of partitions created when reading data is 200. This can be [tuned as need](https://docs.ray.io/en/master/data/performance-tips.html#tuning-read-parallelism). 

In [10]:
arrow_ds = ray.data.from_items(items)
arrow_ds

Dataset(num_blocks=200, num_rows=800000, schema={id: int64, ssn: null, name: null, amount: double, interest: double, state: string, marital_status: string, property: string, dependents: int64, defaulted: int64, gender: string})

In [11]:
arrow_ds.count()

800000

In [12]:
arrow_ds.take(1)

[ArrowRow({'id': 1,
           'ssn': None,
           'name': None,
           'amount': 1.5,
           'interest': 0.30000000000000004,
           'state': 'OR',
           'marital_status': 'single',
           'property': 'condo',
           'dependents': 5,
           'defaulted': 0,
           'gender': 'F'})]

In [13]:
arrow_ds.schema()

id: int64
ssn: null
name: null
amount: double
interest: double
state: string
marital_status: string
property: string
dependents: int64
defaulted: int64
gender: string

### Saving datasets and reading as a parquet files 🗃
Ray datasets support myriad data formats. Let's save this dataset as a parquet file and create `N` partitions, where N=5.

In [14]:
path = os.path.abspath("data_homeowners/interest.parquet")
arrow_ds.repartition(5).write_parquet(path)

Write Progress: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 21.38it/s]


In [15]:
!ls -l data_homeowners/interest.parquet

total 43248
-rw-r--r--  1 jules  staff  2207880 Jul 28 14:54 95a04701700d44e59d254d20c3c7c983_000000.parquet
-rw-r--r--  1 jules  staff  2189237 Jul 28 14:54 95a04701700d44e59d254d20c3c7c983_000001.parquet
-rw-r--r--  1 jules  staff  2187495 Jul 28 14:54 95a04701700d44e59d254d20c3c7c983_000002.parquet
-rw-r--r--  1 jules  staff  2186979 Jul 28 14:54 95a04701700d44e59d254d20c3c7c983_000003.parquet
-rw-r--r--  1 jules  staff  2287108 Jul 28 14:54 95a04701700d44e59d254d20c3c7c983_000004.parquet
-rw-r--r--  1 jules  staff  2207885 Jul 28 20:35 d7343b180f5347ba9dc5981784abd5ff_000000.parquet
-rw-r--r--  1 jules  staff  2189213 Jul 28 20:35 d7343b180f5347ba9dc5981784abd5ff_000001.parquet
-rw-r--r--  1 jules  staff  2187527 Jul 28 20:35 d7343b180f5347ba9dc5981784abd5ff_000002.parquet
-rw-r--r--  1 jules  staff  2186982 Jul 28 20:35 d7343b180f5347ba9dc5981784abd5ff_000003.parquet
-rw-r--r--  1 jules  staff  2287107 Jul 28 20:35 d7343b180f5347ba9dc5981784abd5ff_000004.parquet


In [16]:
arrow_ds = ray.data.read_parquet(path)

In [17]:
arrow_ds.take(1)

[ArrowRow({'id': 1,
           'ssn': None,
           'name': None,
           'amount': 1.5,
           'interest': 0.1,
           'state': 'TX',
           'marital_status': 'single',
           'property': 'house',
           'dependents': 1,
           'defaulted': 0,
           'gender': 'U'})]

### Transforming data with simple methods

Ray datasets support transformation in parallel using `map`. It uses Ray tasks to execute eagerly or synchronously. Among others [transformations](https://docs.ray.io/en/latest/data/package-ref.html#dataset-api), it supports`filter`, `flat_map`, `groupBy`etc.

Let's try a using `.map()`, `.filter()` and `.groupBy` on our dataset. 

The `map()` and `filter()` are row-based operations. This can be expensive for large datasets. However, you can use `map_batches(...)` with batch_size=4096 as default. This will create a task per block and each batch will be vectorized and executed in parallel. Ray tasks are created per block for a map operation. 

Let's try first with row-based transformation

In [18]:
%%time
arrow_ds.filter(lambda x: x['amount'] > 10000).take(1)

Read->Filter: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.89it/s]

CPU times: user 41.7 ms, sys: 26.4 ms, total: 68.1 ms
Wall time: 5.29 s


[ArrowRow({'id': 6667,
           'ssn': None,
           'name': None,
           'amount': 10000.5,
           'interest': 0.1,
           'state': 'OR',
           'marital_status': 'divorced',
           'property': 'rental',
           'dependents': 3,
           'defaulted': 1,
           'gender': 'U'})]

Let's try a `.map_batches()`, which is vectorized. We should expect faster execution. 

*Question: Why the `.map()` returned `ArrowRow` and `.map_batches` returned `PandasRow`*?

Because `map_batch(..., batch_format='native')` promotes it to Pandas DataFrame. You can
promote it to [other formats](https://docs.ray.io/en/master/data/package-ref.html#ray.data.Dataset.map_batches) such as `pyarrow`. The default is `native`, which is pandas.

In [19]:
%%time
arrow_ds.map_batches(lambda df: df[df["amount"] > 10000]).take(1)

Read->Map_Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 10.06it/s]

CPU times: user 47 ms, sys: 18.6 ms, total: 65.6 ms
Wall time: 1.02 s


[PandasRow({'id': 6667,
            'ssn': None,
            'name': None,
            'amount': 10000.5,
            'interest': 0.1,
            'state': 'OR',
            'marital_status': 'divorced',
            'property': 'rental',
            'dependents': 3,
            'defaulted': 1,
            'gender': 'U'})]

You can see that `.map_batches()` is a lot faster than row based. So for large datasets use 
`.map_batches()`.

Let's try a filter operation: both per row operation and per block as vectorized

In [20]:
%%time
arrow_ds.filter(lambda x: x['amount'] > 10000.00 and x['state'] == 'CA').take(2)

Read->Filter: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  4.67it/s]

CPU times: user 25.7 ms, sys: 12.7 ms, total: 38.4 ms
Wall time: 2.15 s


[ArrowRow({'id': 6683,
           'ssn': None,
           'name': None,
           'amount': 10024.5,
           'interest': 0.4,
           'state': 'CA',
           'marital_status': 'undeclared',
           'property': 'cottage',
           'dependents': 2,
           'defaulted': 1,
           'gender': 'M'}),
 ArrowRow({'id': 6691,
           'ssn': None,
           'name': None,
           'amount': 10036.5,
           'interest': 0.30000000000000004,
           'state': 'CA',
           'marital_status': 'domestic',
           'property': 'house',
           'dependents': 1,
           'defaulted': 1,
           'gender': 'U'})]

In [21]:
%%time
arrow_ds.map_batches(lambda df: df[[df["amount"] > 10000] and df["state"] == "CA"]).take(3)

Read->Map_Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 47.75it/s]

CPU times: user 31.2 ms, sys: 11.4 ms, total: 42.6 ms
Wall time: 217 ms


[PandasRow({'id': 2,
            'ssn': None,
            'name': None,
            'amount': 3.0,
            'interest': 0.1,
            'state': 'CA',
            'marital_status': 'undeclared',
            'property': 'cottage',
            'dependents': 4,
            'defaulted': 0,
            'gender': 'U'}),
 PandasRow({'id': 4,
            'ssn': None,
            'name': None,
            'amount': 6.0,
            'interest': 0.30000000000000004,
            'state': 'CA',
            'marital_status': 'divorced',
            'property': 'rental',
            'dependents': 2,
            'defaulted': 0,
            'gender': 'U'}),
 PandasRow({'id': 10,
            'ssn': None,
            'name': None,
            'amount': 15.0,
            'interest': 0.2,
            'state': 'CA',
            'marital_status': 'undeclared',
            'property': 'rental',
            'dependents': 5,
            'defaulted': 0,
            'gender': 'F'})]

Use `groupBy` state and compute the count

In [22]:
results = arrow_ds.groupby("state").count()

Shuffle Reduce: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 1507.60it/s]


In [23]:
results.show()

{'state': 'AZ', 'count()': 200373}
{'state': 'CA', 'count()': 199505}
{'state': 'NM', 'count()': 200054}
{'state': 'NV', 'count()': 200293}
{'state': 'OR', 'count()': 199598}
{'state': 'TX', 'count()': 199659}
{'state': 'UT', 'count()': 200224}
{'state': 'WA', 'count()': 200294}


Get the max of certain columns

In [24]:
results = arrow_ds.max(["amount", "interest", "dependents"])
results

Shuffle Reduce: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 227.17it/s]


ArrowRow({'max(amount)': 1200000.0,
          'max(interest)': 0.5,
          'max(dependents)': 5})

### Accessing datasets using batches or iterating by rows

Datasets can be passed to Ray tasks or actors and iterated over with `.iter_batches()` or `.iter_rows()`. This does not incur a copy, since the blocks of the Dataset are passed by reference as Ray objects. Splitting data as shards and passing to individual Ray Actors to process shards is a common Ray pattern used in distributed training with Ray actors.

Let's examine how we can process a list of shards with a `BatchWorker` Actor  in a distributed fashion

<img src="images/batch_worker.jpg" width="80%" height="35%">

A Ray actor `BatchWorker` working through shards in a batch size of 1024.

In [25]:
@ray.remote
class BatchWorker:
    def __init__(self, rank):
        self.rank = rank         # this could be rank of CPU/GPU or worker id
        self.processed = 0       # how much was processed
    
    @ray.method(num_returns=2)   # we want to return a tuple
    def process_shard_list(self, shard: ray.data.Dataset) -> tuple:
        for batch in shard.iter_batches(batch_size=1024):
            # here you could do something with the batch such as feature
            # preprocessing, minor transformation and then
            # save as a parquet file 
            self.processed = self.processed + len(batch)
        # return items processed, worker id
        return (self.processed, self.rank)     

#### Create batch workers as Ray actors
Each actor will get a shard, list of rows, to work on. We split
our dataset `arrow_ds` into five shards. Each `BatchWorker` gets a shard.

`.split`() splits shards across these batch of workers by using the `locality_hints`.

In [26]:
# create five actors as BatchWorker
batch_workers = [BatchWorker.remote(i) for i in range(1, 6)]

#split into five shards, each one for an actor
shards = arrow_ds.split(len(batch_workers), locality_hints=batch_workers)

print(f"Shard row: {shards[0]}")
print(f"Number of shards:{len(shards)}")
print(f"Number of shard workers:{len(batch_workers)}")

Read progress: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 123.02it/s]

Shard row: Dataset(num_blocks=2, num_rows=320000, schema={id: int64, ssn: null, name: null, amount: double, interest: double, state: string, marital_status: string, property: string, dependents: int64, defaulted: int64, gender: string})
Number of shards:5
Number of shard workers:5


### Launch `BatchWorker` actors

Process each shard. Each `BatchWorker.process_shard_list()` returns a object RefID with a tuple as its value. What we get from this comprehension is a list objectRefs as tuples.

In [27]:
object_refs = [w.process_shard_list.remote(s) for w, s in zip(batch_workers, shards)]
object_refs, len(object_refs)

([[ObjectRef(fc8c7156f41551baf2753dc55984de38374434560100000001000000),
   ObjectRef(fc8c7156f41551baf2753dc55984de38374434560100000002000000)],
  [ObjectRef(a663af0e78d41c9b93ad4a3adae1cd5b6b38b1d80100000001000000),
   ObjectRef(a663af0e78d41c9b93ad4a3adae1cd5b6b38b1d80100000002000000)],
  [ObjectRef(58623fc2e4471617046438ec5ab4a7c7b5a49df90100000001000000),
   ObjectRef(58623fc2e4471617046438ec5ab4a7c7b5a49df90100000002000000)],
  [ObjectRef(1c1ddc6e66f84651e6a37a93d88940066e3468fd0100000001000000),
   ObjectRef(1c1ddc6e66f84651e6a37a93d88940066e3468fd0100000002000000)],
  [ObjectRef(ca64000975ac685e6c77c8dc775eea5d1045918f0100000001000000),
   ObjectRef(ca64000975ac685e6c77c8dc775eea5d1045918f0100000002000000)]],
 5)

Fetch the values from the returned list of ObjectRefs, which is a tuple of (batch_size, worker_rank).

In [28]:
values = [ray.get(ref) for ref in object_refs]
values

[[320000, 1], [320000, 2], [320000, 3], [320000, 4], [320000, 5]]

### Creating and using Ray dataset pipelines

What are dataset pipelines and how they are different from Ray datasets? 

Datasets perform transformation or operations eagerly or synchronously, whereas [DataPipelines](https://docs.ray.io/en/latest/data/package-ref.html#datasetpipeline-api) can execute in an overlapped pipeline executions. For example, if you had operations that require reading from file, transforming data, and then doing some minor feature engineering, these operations can be executed in a normal pipeline fashion. This allows for the overlapped execution of data input (e.g., reading files), computation (e.g. feature preprocessing), and training (e.g., distributed ML training). 

<img src="images/pipeline_window.jpg" width="70%" height="35%">

A `DatasetPipeline` can be constructed in two ways: either by pipelining the execution of an existing Dataset (via `Dataset.window`) or generating repeats of an existing Dataset (via `Dataset.repeat`). 

Let's have a go at it and see what we can do with our simple and synthetic data from above.


### Using Dataset.window

Create simple functions or operations to be executed in a overlapped manner in the pipeline. These functions are simple to illustrate a point. But they can be complex for a particular use case.

In [29]:
def divide_row_value(row, n) -> int:
    return round(row / n)

In [30]:
def double_row_value(row, n) -> int:
    return row * n

In [31]:
def modulo_row_value(row , n) -> int:
    return row % random.randint(1, n)

#### Create a window based pipeline
With a each window of 50 blocks. 

_Question for clarification_:
 * _why num_stages = 2 when I have not created any stages yet?_
 * _what are those stages?_
 * _how is the number of stages determined_?

In [32]:
# Use our original simple dataset from above with 1K rows in integer
ds_pipe = ds.window(blocks_per_window=50)
ds_pipe

DatasetPipeline(num_windows=1, num_stages=2)

### Applying transforms to pipelines adds more pipeline stages.

In [33]:
ds_pipe = ds_pipe.map(lambda row: divide_row_value(row, 2))
ds_pipe = ds_pipe.map(lambda row: double_row_value(row, 3))
ds_pipe = ds_pipe.map(lambda row: modulo_row_value(row, 4))
print(ds_pipe)

DatasetPipeline(num_windows=1, num_stages=5)


#### Iterate our pipeline

 * _Questions for clearification_:
     * _if the `num_stages=5`, why am I seeing only stage 0 and 1 in the output of stages?_

In [34]:
results=[]
for row in ds_pipe.iter_rows():
    results.append(row)
# print(f"Results from each pipeline map function:{results}")
print(f"Total value of the results: {sum(results)}")

Stage 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 38.63it/s]

Total value of the results: 354


Let's try a `Datapipeline` with our synthetic data *Homewowners*

In [35]:
# count or return based on the condition
def count_state(row, state) -> int:
    return 1 if row['state'] == state and row["defaulted"] else 0

In [36]:
arrow_ds_pipe = arrow_ds.window(blocks_per_window=50)
arrow_ds_pipe

DatasetPipeline(num_windows=1, num_stages=2)

In [37]:
arrow_ds_pipe = arrow_ds_pipe.map(lambda row: count_state(row, "CA"))
arrow_ds_pipe

DatasetPipeline(num_windows=1, num_stages=3)

In [38]:
results=[]
for row in arrow_ds_pipe.iter_rows():
    results.append(row)
print(f"Total rows for CA state and defaulted loans rows: {sum(results)}")

Stage 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.79s/it]

Total rows for CA state and defaulted loans rows: 99565


## Ingesting data into Model Trainers
Let's define a toy `Trainer` actor that takes our synthetic data and trains the model and returns loss for that trainer. This is a common pattern in Ray for distributing data to Trainers in a Ray cluster.

<img src="images/trainer_worker.jpg" width="75%" height="40%">

In [39]:
# Our dummy model
def model(input):
    return random.uniform(0, 1)

@ray.remote
class Trainer:
    def __init__(self, rank, model):
        self.rank = rank
        self.model = model
        self.loss = 0.0
        
    def train(self, shard:ray.data.Dataset) -> float:
        for epoch in range(1,21):
            for batch in shard.iter_batches(batch_size=1024):
                output = self.model(batch)
                self.loss = output 
            if epoch % 5 == 0:
                print(f'rank: {self.rank} epoch: {epoch}, loss: {self.loss:.3f}')
        return self.loss

#### Create five trainers, each with a copy of the model and each training on its respective shard

In [40]:
trainers = [Trainer.remote(i, model) for i in range(1, 6)]
trainers

[Actor(Trainer, 09a658d5e2add293a09180b601000000),
 Actor(Trainer, 026ca63a8e6ce714706aba8301000000),
 Actor(Trainer, 061784ec599e8b922871e1a301000000),
 Actor(Trainer, 50fafd4694afaaab30dbe8ff01000000),
 Actor(Trainer, 59143f34016ab15cfe94a6c101000000)]

#### Split the shards across all trainers

In [41]:
shards = arrow_ds.split(n=len(trainers), locality_hints=trainers)
shards

Read progress: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 96.53it/s]


[Dataset(num_blocks=2, num_rows=320000, schema={id: int64, ssn: null, name: null, amount: double, interest: double, state: string, marital_status: string, property: string, dependents: int64, defaulted: int64, gender: string}),
 Dataset(num_blocks=2, num_rows=320000, schema={id: int64, ssn: null, name: null, amount: double, interest: double, state: string, marital_status: string, property: string, dependents: int64, defaulted: int64, gender: string}),
 Dataset(num_blocks=2, num_rows=320000, schema={id: int64, ssn: null, name: null, amount: double, interest: double, state: string, marital_status: string, property: string, dependents: int64, defaulted: int64, gender: string}),
 Dataset(num_blocks=2, num_rows=320000, schema={id: int64, ssn: null, name: null, amount: double, interest: double, state: string, marital_status: string, property: string, dependents: int64, defaulted: int64, gender: string}),
 Dataset(num_blocks=2, num_rows=320000, schema={id: int64, ssn: null, name: null, amount

#### Launch our trainers in a distributed fashion

This will run across the cluster. Check the dashbard to see five actors launched. On a cluster, they will be on five different nodes, whereas on a single node on  
five different cores.

In [42]:
object_refs = [t.train.remote(s) for t, s in zip(trainers, shards)]

In [43]:
ray.get(object_refs)

(Trainer pid=58746) rank: 2 epoch: 5, loss: 0.580
(Trainer pid=58747) rank: 3 epoch: 5, loss: 0.898
(Trainer pid=58748) rank: 4 epoch: 5, loss: 0.646
(Trainer pid=58749) rank: 5 epoch: 5, loss: 0.589
(Trainer pid=58745) rank: 1 epoch: 5, loss: 0.291
(Trainer pid=58746) rank: 2 epoch: 10, loss: 0.526
(Trainer pid=58747) rank: 3 epoch: 10, loss: 0.747
(Trainer pid=58748) rank: 4 epoch: 10, loss: 0.087
(Trainer pid=58749) rank: 5 epoch: 10, loss: 0.872
(Trainer pid=58745) rank: 1 epoch: 10, loss: 0.234
(Trainer pid=58746) rank: 2 epoch: 15, loss: 0.885
(Trainer pid=58747) rank: 3 epoch: 15, loss: 0.313
(Trainer pid=58748) rank: 4 epoch: 15, loss: 0.302
(Trainer pid=58749) rank: 5 epoch: 15, loss: 0.677
(Trainer pid=58745) rank: 1 epoch: 15, loss: 0.849


[0.03370959965278142,
 0.680409357129285,
 0.9705208733174904,
 0.8412062364820266,
 0.62420802704546]

In [44]:
ray.shutdown()

### Exercises
 1. Write some simple transformers, filters, and aggregators with our synthetic data. For example:
  * use [`.add_column()`](https://docs.ray.io/en/master/data/package-ref.html) to add an `age` column
  * filter by gender == 'U'
  * aggregate (or groupby `property`) and count each. 
 2. Add additional pipleline stages function `def count_tx(...)` with our synthetic data. For example, count all people in state of `TX`, `married` and `defaulted`.

### Homework

So far we have covered the basics of Ray Datasets. There are advanced topics that you can now explore since you know the basics. Below is a list of tasks you will want to work through at home.

1. Work through the [NYC example tutorial](extra/ray_data_nyc.ipynb). This explores how you use `.map_batches()` for filtering and map operations using vectorized UDFs
2. Peruse the user guides for advanced examples in [data transformation](https://docs.ray.io/en/master/data/transforming-datasets.html#transforming-datasets) and [ML preprocessing](https://docs.ray.io/en/master/data/dataset-ml-preprocessing.html#datasets-ml-preprocessing)
3. Read how to do large scale [ML ingest](https://docs.ray.io/en/master/data/examples/big_data_ingestion.html)
4. Advanced [pipeline usage](https://docs.ray.io/en/latest/data/advanced-pipelines.html#)

### References

1. [Ray Data Documentation](https://docs.ray.io/en/latest/data/dataset.html)
2. [Ray Data Webinar Talk](https://www.anyscale.com/events/2022/02/23/ray-datasets-scalable-data-preprocessing-for-distributed-ml)